In [1]:
import numpy as np
import scipy as sp
import pywt

# Undersampling

For the various undersampling methods, there is not really much to say, they mostly work like they should. I'll just mention some small things I noticed.

- The functions usually take an array argument, but only use its shape. It's clearer in that case to only accept a shape as argument.
- The functions randomly select a fixed percentage of points / lines. That should be configurable via a function argument as well. You can of course use a default value for that if you want to.

  So the function definition should look like
    ~~~ python
    def randomline(shape, percentage=0.33):
        # [...]
    ~~~

- The functions construct complex arrays. That's not really needed, `dtype=int` (or even `dtype=bool`) is enough here.
- The `import math` inside the function body is unusual. That should probably be done once at the top of the file, together with all the other imports. Also, I assume you use `math.floor` instead of `np.floor` since the latter returns a `float`, not an `int`. But simply `int(0.33 * x.shape[1])` would have been enough here.
- The `spiral` function looks too complicated. E.g. why does it need a nested function and a construction like this:

    ~~~ python
    correction=1
    if size!=64:
        correction = size/64 
    ~~~

  Also, it's really slow. There's long section on performance of the (T)SPR below, so I'll go into performance stuff here again, but I think a lot of this can be vectorized nicely. Maybe you can try if you want to.
- Using `np.random.seed` inside a function is not a good idea. I understand that you want to produce reproducible results, but the seed is a *global* value for the `np.random` functions, so calling randomline also sets the seed for everything else. Also, the choice whether to use (pseudo-)random or “fixed-random” numbers should be left to the caller.

  `np.random` has a new-ish [random number generator interface](https://numpy.org/doc/stable/reference/random/generator.html) that does not rely on a *global* seed, but stores all state in a concrete generator object that can be passed around explicitly. Drawing samples from various distributions is then done via methods exposed by this object. It may be good style for functions that want to generate random numbers to accept a random generator object as argument. A random number generator can be created using `np.random.default_rng()`, which accepts an optional `seed` parameter. The important part is that the caller decides which rng to use, and may even use multiple independent rngs, or use a fixed-seeded one for testing and an “unseeded“ one for numerical simulations.

In [2]:
def randomline(rng, shape, percentage=0.33):
    pattern = np.zeros(shape, dtype=bool)
    # Btw. high=shape[0], not shape[0]-1. Python convention for ranges is to omit
    # the high index itself, so integers are in the range between low and high-1 (inclusive).
    for j in rng.integers(low=0, high=shape[0], size=int(percentage * shape[1])):
        pattern[:, j] = 1
    return pattern

In [3]:
rng = np.random.default_rng()
randomline(rng, (6, 6), percentage=0.5)

array([[ True, False,  True, False, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False,  True],
       [ True, False,  True, False, False,  True]])

In [4]:
# You can construct another pattern with the same rng
randomline(rng, (4, 4), percentage=0.5)

array([[ True, False, False,  True],
       [ True, False, False,  True],
       [ True, False, False,  True],
       [ True, False, False,  True]])

In [5]:
# Or you an use a completely new, maybe preseeded rng
seeded_rng = np.random.default_rng(seed=11)
randomline(seeded_rng, (4, 4), percentage=0.5)

array([[ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False]])

- As illustrated here, some of the functions don't handle the case of some points / lines being selected multiple times. Therefore, the example above only has one line, but there should be two.

# Wavelets

While the original idea was to first handle the simpler case of the identity as a sparsity transform, it's great to see that you also looked into wavelet transforms and their implementation in Python. A lot of the code is commented out in the notebook, so I assume you're experimenting with it. Just one remark: in the partially commented code, you apply the subsampled FFT to the wavelet coefficients.

~~~ python
img = cv2.imread("brain.jpeg",0)
C =pywt.wavedec2(img,"db5",mode="periodization",level=None)
arr,coeff_slices = pywt.coeffs_to_array(C)
original = np.fft.fft2(arr)
center_ = np.fft.fftshift(original)
gauss_lines = gaussline(center_)
b = center_*gauss_lines
~~~

That's the wrong way around. You apply the both the subsampled Fourier transform and the wavelet transform to the image, and the reconstruction is performed such that the former matches the data while the latter is sparse: 

$$
\text{Minimize} \quad \lVert \mathcal{F}(x) - y \rVert_2^2 + \alpha \lVert \mathcal{W}(x) \rVert_1 \quad \text{w.r.t. $x$}
$$

Alternatively, you can reconstruct in wavelet space and demand that the coefficients are sparse while the subsampling of the Fourier transform of the *inverse* wavelet transform of the coefficients matches the data:

$$
\text{Minimize} \quad \lVert \mathcal{F}(\mathcal{W}^*(z)) - y \rVert_2^2 + \alpha \lVert z \rVert_1 \quad \text{w.r.t. $z$}
$$

# ISTA

I think there are some errors in you ISTA implementations. You effectively implemented:

$$
x_0 = \mathcal{F}^*(y), \qquad
x_{k+1} = \operatorname{Threshold}\left(x_k + \alpha^{-1}\mathcal{F}^*(y - \mathcal{F}(x))\right)
$$

where the data $y$ are computed as

$$
y = \operatorname{Subsample}(\mathcal{F}(\mathrm{img})).
$$

The issue here is that $\mathcal{F}$ is the full Fourier transform, so you effectively solve

$$
\mathcal{F}(x) = \operatorname{Subsample}(\mathcal{F}(\mathrm{img})) \quad (\text{+ sparsity constraint}).
$$

This demands that the Fourier transform of your reconstruction is actually close to zero at the points where there's no data and may be the cause for the pretty heavy “mirror artifacts” that you see.

In the wavelet case, you do use the sampling pattern via `extract_pattern`. Btw, `extract_pattern(y)` can simply be written as `(y != 0)`, you don't really need the helper function. Also, it should be performed only once outside the loop for efficiency. Finally, this construct should not be used at all. Extracting the pattern from the data is neat, but there may be accidental zeros in the data that you do not want to project out this way. If your data are `floats`, that may be improbable enough to ignore, but MRI datasets often don't consists of floating point measurements. Instead, they may be given as e.g. 256-grayscale-images encoded as `ints`, and in that case you have a non-vanishing probabilty of an exact zero in the data. So the pattern should be passed in as an additional parameter.

Your wavelet ISTA does not, however, do wavelet thresholding. You compute the wavelet coefficients in `arr`, but only use it for evaluating the objective function (which does not actually enter the iteration). You still perform the thresholding in image space.

Finally, as a minor detail, the $L^1$- and $L^2$-norms in the objective can be computed like this:

~~~ python
J[k] = np.linalg.norm(Hx - y)**2 + lamb * np.linalg.norm(arr.ravel(), 1)
~~~

instead of

~~~ python
J[k] = np.sum(np.abs(Hx-y)**2)+lamb*np.sum(np.abs(arr))
~~~

Both `np.sum` and `np.abs` can be bit slow in my experience, and using `np.linalg.norm` for computing a norm is clearer anyway. `.ravel()` is needed to reshape `arr` into a 1d-array, since the `1`-norm is only computed along the `0`-axis for 2d arrays by default.

# FISTA

FISTA (Fast ISTA) is a simple update of the ISTA method that improves convergence of the objective from $\mathcal{O}(1/n)$ to $\mathcal{O}(1/n^2)$. It involves a simple additional relaxation step and a somewhat “magical” stepsize rule due to Nesterov. Maybe you can also try implementing that method.  There is a [short paper](https://doi.org/10.1109/ICASSP.2009.4959678) by Beck and Teboulle that describes the method (I'll also attach it to my email in case you can't download it).

The paper only mentions the identity case, but the wavelet case works just like for the ISTA method. Also, it contains a method with backtracking for forward operators with unknown Lipschitz constant. For us, that's not relevant, as $\mathcal{F}$ and $\mathcal{W}$ are unitary and the the subsampling has operator norm 1, so you can just use the non-backtracking version.

# Conjugate gradient

I only briefly looked at this part, but it looks like a pretty direct translation of the paper. I think there's an error in `delta_fm` regarding the matrix `W`. The paper says to compute (with $z := \Psi m$)

$$
W^{-1} z = \left( \frac{z_i}{\sqrt{|z_i|^2 + \mu}} \right)_i.
$$

But due to the fact that in the code, $z$ (i.e. `arr`) is a 2d array, you apparently mixed that up a bit to

$$
\left( \frac{z_{ij}}{\sqrt{|z_{ii}|^2 + \mu}} \right)_{ij}.
$$

The code should simply be

~~~ python
arr / np.sqrt(np.abs(arr)**2 + 1e-11)
~~~

No need to construct the matrix `W` at all.

There are also some smaller details, but I'll leave them out for now, except these two: 

- Even if the code in `delta_fm` was correct, you should not use `np.linalg.inv`. It would be easier to construct the inverse directly by simply inverting the diagonal. You never know what `inv` does; it may be clever enough to figure out that the matrix is diagonal, or it may use some more expensive general-purpose inversion method. In general, you should always think twice before calling `inv`. It's almost never the right thing to do. E.g. even if you didn't have a nice explicit formula for the inverse of `W`, all you want to do is to apply it to `arr`, so `np.linalg.solve(W, arr)` would be the faster and more stable choice.
- In the line search loop, all non-`t`-dependent terms should be precomputed, especially `f_cost(m, data, lamb)`, which is rather expensive.

Maybe you can try the reconstructions again with the fixed `delta_fm`. The `randomgauss` is not too bad as it stands, but not very good either. Do you have an idea where the horizontal artifact around image line 450 comes from?

If the results don't look good, maybe try with more sampling lines first, or better yet a fully sampled k-space. That just has to work if there are no other errors, so it's a good first check.

Maybe you also need to play with the regularization parameter, the tolerance or the number of iterations. It can help to print e.g. the cost function and the step size picked by the line search to see if the method makes any progress at all.

Maybe it will also turn out that the TV norm is relevant after all and can't be omitted, as we currently do. But I would not look into that direction right away.

# Compressed SENSE

That part looks interesting on first sight, but I did not really look into it. It's a pretty different approach. If you want to look into this direction to use it as a comparison to the variational approach, that's ok of course, but I can't tell you anything specific about it.

# Performance of the (T)SPR functions

You remarked that the (T)SPR functions take forever to compute. Maybe we can look a bit into why that is. We won't be able to make it significantly faster, but the attemtp is kind of illustrative regardless. The underlying problem is that, for $n \times n$ images, you need to perform $\mathcal{O}(n^2)$ FFTs, each with $\mathcal{O}(n^2 \log(n))$ complexity, so you end up with $\mathcal{O}(n^4 \log(n)^2)$. It simply does not scale nicely.


> **Note:** This part grew rather long, which is a common tendency when doing performance optimization; there's just too much to look at. I'll still leave it in, as some of this can be useful generally. But since it's not the main focus of your project, feel free to skip it. It's the rest of the notebook, although a lot of it is copy-pasted code with small modifications and profiler output.

This is basically the function I sent you some time ago. I'll only talk about the SPR, but most of it should be applicable to the TSPR as well.

In [6]:
def spr(gridsize, sampling_pattern):
    maxima = np.zeros(gridsize,dtype = np.complex)
    for x in range(gridsize[0]):
        for y in range(gridsize[1]):
            e_i = np.zeros(gridsize,dtype = np.complex)
            e_i[x, y] = 1
            psf_i = apply_Fu_adjoint(sampling_pattern, apply_Fu(sampling_pattern, e_i))
            psf_i = psf_i / psf_i[x, y]
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

First of all, this contains an error: it's missing the absolute value (my fault). Also, passing in the grid size is not really necessary as it can be obtained from the sampling pattern. Finally, I'll inline the `apply_Fu` and `apply_Fu_adjoint` functions for simplicity.

In [7]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * np.fft.fftshift(np.fft.fft2(e_i))
            psf_i = np.fft.ifft2(np.fft.ifftshift(pattern * ft))
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

Let's also create a random pattern.

In [8]:
def make_pattern(rng, shape, n):
    pattern = np.zeros(shape)
    while n != 0:
        x, y = rng.integers(shape)
        if pattern[x, y] == 0:
            pattern[x, y] = 1
            n = n - 1
    return pattern

# I'll recreate the rng here in case you only want to run this cell.
# Usually, that would not be needed, as we already created an rng further up
# in the notebook.
rng = np.random.default_rng(seed=123)
# Use shape (64, 64) since, as you know, the code takes forever
pattern = make_pattern(rng, (64, 64), 1500)

In [9]:
spr(pattern)

0.05752249207416043

## Profiling code in Jupyter

There are multiple ways to investigate the performance of code in Jupyter.

### `%time`, `%timeit`

These are the easiest options. `%time` simply measures how long a command takes.

In [10]:
%time spr(pattern)

CPU times: user 1.41 s, sys: 11.7 ms, total: 1.42 s
Wall time: 1.42 s


0.05752249207416043

About 1.4s (on my machine) is really rather slow. `%time` only measures a single run of the command. `%timeit` performs multiple runs for increased accuracy. Slow commands are run less often to keep total time low.

In [11]:
%timeit spr(pattern)

1.39 s ± 7.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `%prun`

Pythons built-in profiler can be invoked via `%prun`. It opens a popup window that can be a bit hard to read; there's a button to open the result in a new tab in the top-right corner. I'll copy-paste the output below.

In [12]:
%prun spr(pattern)

~~~
         667661 function calls (610317 primitive calls) in 1.608 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.502    0.502    1.608    1.608 <ipython-input-7-e6a7121f84b8>:1(spr)
    16384    0.411    0.000    0.411    0.000 {built-in method numpy.fft._pocketfft_internal.execute}
     8192    0.193    0.000    0.234    0.000 numeric.py:1110(roll)
77825/20481    0.055    0.000    1.079    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    16384    0.040    0.000    0.499    0.000 _pocketfft.py:51(_raw_fft)
     8192    0.036    0.000    0.722    0.000 _pocketfft.py:649(_raw_fftnd)
     8192    0.034    0.000    0.134    0.000 _pocketfft.py:630(_cook_nd_args)
    49152    0.032    0.000    0.032    0.000 {built-in method numpy.array}
     4097    0.031    0.000    0.031    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     8192    0.023    0.000    0.066    0.000 fromnumeric.py:42(_wrapit)
    24576    0.021    0.000    0.099    0.000 fromnumeric.py:55(_wrapfunc)
     8192    0.020    0.000    0.020    0.000 {method 'take' of 'numpy.ndarray' objects}
     4096    0.011    0.000    0.139    0.000 helper.py:79(ifftshift)
     4096    0.011    0.000    0.150    0.000 helper.py:22(fftshift)
     8192    0.011    0.000    0.020    0.000 numeric.py:1277(normalize_axis_tuple)
    32768    0.010    0.000    0.010    0.000 {built-in method numpy.core._multiarray_umath.normalize_axis_index}
    40960    0.009    0.000    0.039    0.000 _asarray.py:16(asarray)
    16384    0.009    0.000    0.043    0.000 <__array_function__ internals>:2(swapaxes)
     4097    0.009    0.000    0.043    0.000 fromnumeric.py:73(_wrapreduction)
     8192    0.008    0.000    0.264    0.000 _pocketfft.py:192(ifft)
     4097    0.008    0.000    0.008    0.000 {built-in method numpy.zeros}
    16384    0.008    0.000    0.008    0.000 {method 'swapaxes' of 'numpy.ndarray' objects}
    16384    0.007    0.000    0.026    0.000 fromnumeric.py:553(swapaxes)
     8192    0.007    0.000    0.258    0.000 _pocketfft.py:98(fft)
     8192    0.007    0.000    0.088    0.000 fromnumeric.py:97(take)
    32768    0.007    0.000    0.007    0.000 {built-in method builtins.getattr}
     4097    0.007    0.000    0.050    0.000 fromnumeric.py:2551(amax)
     8192    0.006    0.000    0.276    0.000 <__array_function__ internals>:2(ifft)
     8192    0.006    0.000    0.269    0.000 <__array_function__ internals>:2(fft)
     8192    0.005    0.000    0.012    0.000 <__array_function__ internals>:2(empty_like)
     8192    0.005    0.000    0.256    0.000 <__array_function__ internals>:2(roll)
     8192    0.005    0.000    0.097    0.000 <__array_function__ internals>:2(take)
     8192    0.005    0.000    0.009    0.000 numeric.py:1327(<listcomp>)
     4097    0.003    0.000    0.057    0.000 <__array_function__ internals>:2(amax)
     8192    0.003    0.000    0.003    0.000 numeric.py:1191(<dictcomp>)
     4096    0.003    0.000    0.146    0.000 <__array_function__ internals>:2(ifftshift)
     4096    0.003    0.000    0.368    0.000 _pocketfft.py:950(ifft2)
     4096    0.003    0.000    0.360    0.000 _pocketfft.py:859(fft2)
     4097    0.003    0.000    0.003    0.000 fromnumeric.py:74(<dictcomp>)
     4096    0.003    0.000    0.373    0.000 <__array_function__ internals>:2(ifft2)
     4096    0.003    0.000    0.366    0.000 <__array_function__ internals>:2(fft2)
    24576    0.003    0.000    0.003    0.000 {built-in method builtins.len}
     4096    0.003    0.000    0.003    0.000 helper.py:117(<listcomp>)
     4096    0.002    0.000    0.156    0.000 <__array_function__ internals>:2(fftshift)
     4096    0.002    0.000    0.002    0.000 helper.py:70(<listcomp>)
    16384    0.002    0.000    0.002    0.000 _pocketfft.py:94(_fft_dispatcher)
    16384    0.002    0.000    0.002    0.000 fromnumeric.py:549(_swapaxes_dispatcher)
     8192    0.002    0.000    0.004    0.000 _asarray.py:88(asanyarray)
    12289    0.002    0.000    0.002    0.000 {method 'items' of 'dict' objects}
     8192    0.001    0.000    0.001    0.000 fromnumeric.py:93(_take_dispatcher)
     8192    0.001    0.000    0.001    0.000 helper.py:18(_fftshift_dispatcher)
     8192    0.001    0.000    0.001    0.000 _pocketfft.py:659(_fftn_dispatcher)
     8192    0.001    0.000    0.001    0.000 multiarray.py:77(empty_like)
     8192    0.001    0.000    0.001    0.000 numeric.py:1106(_roll_dispatcher)
     8192    0.001    0.000    0.001    0.000 {method 'reverse' of 'list' objects}
     4097    0.001    0.000    0.001    0.000 fromnumeric.py:2546(_amax_dispatcher)
        1    0.000    0.000    1.608    1.608 {built-in method builtins.exec}
        1    0.000    0.000    1.608    1.608 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
~~~

The table shows, for each function:
- how often it was called,
- its execution time, both total (`tottime`) and per call (`percall`),
- its cumulative execution time (i.e. including subfunctions), again total (`cumtime`) and `percall`, and
- its name. For functions imported from modules, it shows the file name and line number. Functions defined in Jupyter don't have a proper file name. They are displayed like `<ipython-input-7-e6a7121f84b8>:1(spr)`, which denotes the function `spr` in cell `7`, line `1` (you can toggle line numbers in cells via the `View` menu).

The line

~~~
    16384    0.411    0.000    0.411    0.000 {built-in method numpy.fft._pocketfft_internal.execute}
~~~

already hints that a large part of the time is spent in the interna of NumPy's FFT, which we can't really improve upon (but see below). The next line,

~~~
     8192    0.193    0.000    0.234    0.000 numeric.py:1110(roll)
~~~

does however show a possible improvement. `roll` is the main part of the implementation of `(i)fftshift`; you can also see that in the `cumtime` of the `fftshifts` further down the list. In general, `fftshift` performs a copy of the data and should be avoided in inner loops. It's better to get used to the zero frequency being at `(0, 0)`, and to shift what needs to be shifted before the iteration. 

In [13]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * np.fft.fft2(e_i)
            psf_i = np.fft.ifft2(pattern * ft)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

pattern = np.fft.ifftshift(pattern)  # it was uniform random anyway, but this way we'll get the same result as above

In [14]:
%timeit spr(pattern)

1.1 s ± 8.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Also, since the pattern is `0` or `1` everywhere, we do not need to multiply by it twice:

In [15]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * np.fft.fft2(e_i)
            psf_i = np.fft.ifft2(ft)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

In [16]:
%timeit spr(pattern)

1.1 s ± 42.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


(That didn't actually do much.) Finally, SciPy has its own FFT module that tends to be faster than NumPy's, so let's switch to that.

In [17]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * sp.fft.fft2(e_i)
            psf_i = sp.fft.ifft2(ft)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

In [18]:
%timeit spr(pattern)

889 ms ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The SciPy FFT interface is pretty similar, but it does provide some additional options: `overwrite_x` lets it overwrite the given argument if needed to improve performance. Of course, that is only possible if the argument is not needed anymore afterwards, which is however the case here:

In [19]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * sp.fft.fft2(e_i, overwrite_x=True)
            psf_i = sp.fft.ifft2(ft, overwrite_x=True)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

In [20]:
%timeit spr(pattern)

883 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Well, not every change is worth it. Finally, there's a `workers` options that enables a multi-threaded implementation. Playing with this can be useful, but multi-threading is complex and not always the best solution at this level. `workers=-1` uses one thread per CPU.

In [21]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * sp.fft.fft2(e_i, workers=-1)
            psf_i = sp.fft.ifft2(ft, workers=-1)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

In [22]:
%timeit spr(pattern)

1.15 s ± 192 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That's definitely worse; the overhead is larger than the gain here. But we'll come back to this below.

### `%lprun`

Finally, since `%prun`'s output can be difficult to interpret, there's the line profiler. This is a third-party module, you need to `pip install line-profiler` it first, and then load it into Jupyter via

In [23]:
%load_ext line_profiler

Also, let's go back to the version without `overwrite_x` or `workers`.

In [24]:
def spr(pattern):
    maxima = np.zeros(pattern.shape)
    for x in range(pattern.shape[0]):
        for y in range(pattern.shape[1]):
            e_i = np.zeros(pattern.shape)
            e_i[x, y] = 1
            ft = pattern * sp.fft.fft2(e_i)
            psf_i = sp.fft.ifft2(ft)
            psf_i = np.abs(psf_i / psf_i[x, y])
            psf_i[x, y] = -np.inf
            maxima[x, y] = np.max(psf_i)
    spr = np.max(maxima)
    return spr

Using the line profiler is not totally straight-forward, and its better suited for `.py`-files instead of notebooks, but for really simple tasks, this is all you need:

In [25]:
%lprun -f spr spr(pattern)

Again, this pops up a window containing

~~~
Timer unit: 1e-06 s

Total time: 1.18011 s
File: <ipython-input-57-3a58d5247693>
Function: spr at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def spr(pattern):
     2         1         15.0     15.0      0.0      maxima = np.zeros(pattern.shape)
     3        65         40.0      0.6      0.0      for x in range(pattern.shape[0]):
     4      4160       2663.0      0.6      0.2          for y in range(pattern.shape[1]):
     5      4096      12670.0      3.1      1.1              e_i = np.zeros(pattern.shape)
     6      4096       3325.0      0.8      0.3              e_i[x, y] = 1
     7      4096     383639.0     93.7     32.5              ft = pattern * sp.fft.fft2(e_i)
     8      4096     339847.0     83.0     28.8              psf_i = sp.fft.ifft2(ft)
     9      4096     356987.0     87.2     30.3              psf_i = np.abs(psf_i / psf_i[x, y])
    10      4096       6312.0      1.5      0.5              psf_i[x, y] = -np.inf
    11      4096      74587.0     18.2      6.3              maxima[x, y] = np.max(psf_i)
    12         1         26.0     26.0      0.0      spr = np.max(maxima)
    13         1          1.0      1.0      0.0      return spr
~~~

This shows, for each line of code, how often it was executed, how long it took (total and per hit), and the percentage of the total time spent on that line. Note also that the total execution time is longer than in the `%timeit` above: line profiling adds quite some overhead, so by measuring the timing, you actually change it. As expected, most of the time is spent in the FFTs, but whats also interesting is that about 37% are actually in more boring operations: division, abs and max.

## Vectorization

There are not many straight-forward optimizations to make anymore. The most important problem of the code is the double for loop and the associated indexing operations. Both Python loops and NumPy elementwise operations are *slow*. The first direction to optimize code is generally to try to replace them with vectorized NumPy operations. These are implemented in C or Fortran and avoid much of Python's overhead.

The way to do this here (with a rather strong caveat noted below) is to construct all basis vectors at once, which can conveniently done using `numpy.eye` and some reshaping.

In [26]:
def spr(pattern):
    shape = pattern.shape
    size = pattern.size
    e = np.eye(size).reshape((size,) + shape) # e now is a 3d array, the first dim enumerates the basis vectors
    ft = sp.fft.fft2(e) # fft2 only operates on the last 2 dims
    ft = ft * pattern # pattern is multiplied with last 2 dims via broadcasting
    psf = sp.fft.ifft2(ft).reshape(size, size) # reshape into 2d array, in line with notation in the paper
    psf = np.abs(psf / np.diag(psf)) # np.diag extracts the diagonal (1d), division uses broadcasting
    np.fill_diagonal(psf, -np.inf) # set the diagonal to -inf to ignore the maximum
    return np.max(psf)

In [28]:
%timeit spr(pattern)

781 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Again an improvement, although not by orders of magnitude (which getting rid of Python for-loops can well be). Also, vectorized code can be harder to read and write, and occasionally requires special functions like `fill_diagonal` for otherwise straight-forward constructs like `psf[i, i] = -np.inf`.

To make things not unnecessarily long, I'll just list some further small improvements and show you the end result. You can try them individually if you like:

- Exchange division and `abs`:
    ~~~ python
    psf = np.abs(psf)
    psf = psf / np.diag(psf)
    ~~~
  This avoids complex divisions and is a surprisingly good improvement (given that it's such a small change).
- Avoid `np.abs`. I don't know why, but it's weirdly slow.
    ~~~ python
    psf = np.sqrt(psf.real**2 + psf.imag**2)
    ~~~
  We can even move the sqrt to the `return` line
    ~~~ python
    return np.sqrt(np.max(psf))
    ~~~
  so it's not computed for all `psf` elements. That not much of an improvement, though.
- Prefer in-place operations, they avoid unneeded array allocations (also just a small gain):
    ~~~ python
    ft *= pattern
    # ...
    psf /= np.diag(psf)
    ~~~
- Use `overwrite_x=True` for the FFTs. This time it's a small gain.

In [29]:
def spr(pattern):
    shape = pattern.shape
    size = pattern.size
    e = np.eye(size).reshape((size,) + shape)
    ft = sp.fft.fft2(e, overwrite_x=True)
    ft *= pattern
    psf = sp.fft.ifft2(ft, overwrite_x=True).reshape(size, size)
    psf = psf.real**2 + psf.imag**2
    psf /= np.diag(psf)
    np.fill_diagonal(psf, -np.inf)
    return np.sqrt(np.max(psf))

In [30]:
%timeit spr(pattern)

504 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Finally, let's check multi-threading again:

In [31]:
def spr(pattern):
    shape = pattern.shape
    size = pattern.size
    e = np.eye(size).reshape((size,) + shape)
    ft = sp.fft.fft2(e, overwrite_x=True, workers=-1)
    ft *= pattern
    psf = sp.fft.ifft2(ft, overwrite_x=True, workers=-1).reshape(size, size)
    psf = psf.real**2 + psf.imag**2
    psf /= np.diag(psf)
    np.fill_diagonal(psf, -np.inf)
    return np.sqrt(np.max(psf))

In [32]:
%timeit spr(pattern)

337 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Since we now perform all FFTs in a single run, the amount of work is large enough so that the gain from multiple threads outweighs the cost.

## So?

All in all, the runtime is reduced from 1.4s to 0.3s. That's not bad, but not great either, so for larger images or many different randomly-generatad sampling patterns, you need a lot of patience or better options.

Moreover, the vectorized code has a significant flaw: it generated all basis vectors in memory. That's fine for small images, but quickly infeasible for larger ones. In that case, you'd have to resort to some chunking strategy: generate, say, 1000 basis vectors, process them vectorized, move to the next chunk, etc. That makes the code a lot uglier.

On the bright side, the code is trivially parallelizable. The computations for different basis vectors do not influence each other at all, so it can be scaled to an arbitrary number of CPUs and/or different computers, but that also requires significant code changes. Again, since this is not the focus of the project, I'd not look into that direction.

For pracitcal purposes, I'd say you can simply evaluate the SPR for various sampling strategies on small images and for larger images just pick the strategy that performed best.